In [1]:
import numpy as np
from sklearn.isotonic import IsotonicRegression
from sklearn.linear_model import LogisticRegression
import torch
import matplotlib.pyplot as plt

In [2]:
rand_feature_map = torch.normal(0, 1, (10, 512, 512))

In [3]:
X = rand_feature_map.numpy().reshape(-1)
y = X >= 0
X = np.clip(X + np.random.normal(0, 1, X.shape), 0, 1)

X = X[:1_000]
y = y[:1_000]

In [4]:
lr = LogisticRegression()
lr.fit(X.reshape(-1, 1), y)

LogisticRegression()

In [5]:
ir = IsotonicRegression(out_of_bounds='clip')
ir.fit(X, y)

IsotonicRegression(out_of_bounds='clip')

In [6]:
class TorchLogisticRegression(torch.nn.Module):
    def __init__(self, lr: LogisticRegression):
        super().__init__()
        self._coef = torch.nn.Parameter(torch.from_numpy(lr.coef_))
        self._intercept = torch.nn.Parameter(torch.from_numpy(lr.intercept_))
    def forward(self, probs):
        return torch.sigmoid(probs*self._coef + self._intercept)

In [7]:
class TorchIsotonicRegression(torch.nn.Module):
    def __init__(self, ir: IsotonicRegression):
        super().__init__()
        self.x_vals = torch.nn.Parameter(torch.from_numpy(ir.f_.x), requires_grad=False)
        self.y_vals = torch.nn.Parameter(torch.from_numpy(ir.f_.y), requires_grad=False)
    def forward(self, inputs):
        masks = []
        for x_val, y_val in zip(self.x_vals, self.y_vals):
            masks.append(torch.where(inputs >= x_val, y_val, 0.))
        vals, _ = torch.max(torch.stack(masks, dim=0), dim=0)
        return vals 

In [8]:
torchLR = TorchLogisticRegression(lr)
torchIR = TorchIsotonicRegression(ir)

### LogisticRegression Test

In [9]:
xs = rand_feature_map.numpy()
xs = xs.reshape(-1, 1)

In [10]:
%%timeit
res = lr.predict_proba(xs)[:, 1]
res = res.reshape(rand_feature_map.shape)

27.9 ms ± 812 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
%%timeit
with torch.no_grad():
    res = torchLR.forward(rand_feature_map)
    _ = res.numpy()

4.74 ms ± 94.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [12]:
torchLR.to("cuda")
rand_feature_map = rand_feature_map.to("cuda")

In [13]:
%%timeit
with torch.no_grad():
    res = torchLR.forward(rand_feature_map)
    _ = res.cpu().numpy()

2.35 ms ± 7.18 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### Test Results

In [14]:
torchLR.to("cpu")
rand_feature_map = rand_feature_map.to("cpu")
with torch.no_grad():
    torch_res = torchLR.forward(rand_feature_map)
    torch_res = torch_res.numpy()

In [15]:
xs = rand_feature_map.numpy()
xs = xs.reshape(-1, 1)
sklearn_res = lr.predict_proba(xs)[:, 1]
sklearn_res = sklearn_res.reshape(rand_feature_map.shape)

In [16]:
np.allclose(torch_res, sklearn_res)

True

### IsotonicRegression Test

In [17]:
xs = rand_feature_map.numpy()
xs = xs.reshape(-1)

In [18]:
%%timeit
res = ir.predict(xs)
res = res.reshape(rand_feature_map.shape)

38.7 ms ± 719 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [19]:
%%timeit
with torch.no_grad():
    res = torchIR.forward(rand_feature_map)
    _ = res.numpy()

68 ms ± 1.23 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [20]:
torchIR.to("cuda")
rand_feature_map = rand_feature_map.to("cuda")

In [21]:
%%timeit
with torch.no_grad():
    res = torchIR.forward(rand_feature_map)
    _ = res.cpu().numpy()

6.93 ms ± 12.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### Test Results

In [22]:
torchIR.to("cpu")
rand_feature_map = rand_feature_map.to("cpu")
with torch.no_grad():
    torch_res = torchIR.forward(rand_feature_map)
    torch_res = torch_res.numpy()

In [23]:
xs = rand_feature_map.numpy()
xs = xs.reshape(-1)
sklearn_res = ir.predict(xs)
sklearn_res = sklearn_res.reshape(rand_feature_map.shape)

In [24]:
# formula: y = y1 + (x - x1)*slope

# slope[n] is for xs in interval x[n] - x[n+1]
# so formula is: y = y[n] + (x - x[n])*slopes[n]

In [25]:
rand_feature_map = rand_feature_map.to("cpu")

In [26]:
x_vals = torch.nn.Parameter(torch.from_numpy(ir.f_.x), requires_grad=False)
y_vals = torch.nn.Parameter(torch.from_numpy(ir.f_.y), requires_grad=False)
slopes = torch.nn.Parameter(
    torch.from_numpy(
        np.concatenate([
            np.array([0.]),
            (ir.f_.y[1:] - ir.f_.y[:-1]) / (ir.f_.x[1:] - ir.f_.x[:-1]),
        ])
))
masks = []
for x_val, y_val in zip(x_vals, y_vals):
    masks.append(torch.where(rand_feature_map >= x_val, x_val, 0.))
_, ind = torch.max(torch.stack(masks, dim=0), dim=0)
y = y_vals[ind] + (rand_feature_map - x_vals[ind])*slopes[ind]

In [27]:
ind[0, 32, 32]

tensor(4)

In [46]:
slopes[ind[0, 32, 32]]

tensor(25.3704, dtype=torch.float64, grad_fn=<SelectBackward0>)

In [43]:
(rand_feature_map[0, 32, 32] - x_vals[ind[0, 32, 32]])

tensor(0.0843, dtype=torch.float64)

In [36]:
slopes[ind[0, 32, 32]]

tensor(25.3704, dtype=torch.float64, grad_fn=<SelectBackward0>)

In [32]:
ir.f_(0.1069)

array(0.60576923)

In [29]:
x_vals

Parameter containing:
tensor([0.0000, 0.0075, 0.0076, 0.0185, 0.0227, 0.4487, 0.4644, 0.6429, 0.6581,
        0.8959, 0.8983, 0.9771, 0.9786, 1.0000], dtype=torch.float64)

In [30]:
y_vals

Parameter containing:
tensor([0.2346, 0.2346, 0.5000, 0.5000, 0.6058, 0.6058, 0.6279, 0.6279, 0.6912,
        0.6912, 0.7222, 0.7222, 0.8731, 0.8731], dtype=torch.float64)

In [47]:
slopes

Parameter containing:
tensor([0.0000e+00, 0.0000e+00, 7.7339e+03, 0.0000e+00, 2.5370e+01, 0.0000e+00,
        1.4043e+00, 0.0000e+00, 4.1708e+00, 0.0000e+00, 1.3062e+01, 0.0000e+00,
        1.0328e+02, 0.0000e+00], dtype=torch.float64, requires_grad=True)